In [13]:
import tensorflow as tf
from tensorflow import keras
from keras import regularizers
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D, Activation, BatchNormalization
from keras.preprocessing.image import ImageDataGenerator

import os

In [14]:
import numpy as np 
import matplotlib.pyplot as plt
from PIL import Image

In [15]:
TARGET_SIZE = (100, 100)

## Data Augmentation 

In [16]:
datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=0.2,
    height_shift_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest',
    brightness_range=[0.2, 1.5]
)

### Generator of train images

In [17]:
data = os.path.join(os.getcwd(), 'dataset/processed_split/train')

train_generator = datagen.flow_from_directory(   
    data, 
    target_size=TARGET_SIZE,
    batch_size=32,
    class_mode='categorical',
    color_mode='grayscale'
)

num_batches = len(train_generator)
print("Number of batches: ", num_batches)

Found 1280 images belonging to 16 classes.
Number of batches:  40


### Generator of test images

In [18]:
data = os.path.join(os.getcwd(), 'dataset/processed_split/test')

test_generator = datagen.flow_from_directory(
    data, 
    target_size=TARGET_SIZE,
    batch_size=32,
    class_mode='categorical',
    color_mode='grayscale'
)

num_batches = len(test_generator)
print("Number of batches: ", num_batches)

Found 320 images belonging to 16 classes.
Number of batches:  10


Son 100 imagenes, de 100x100, en los 3 canales RGB.

## Modelo de Datos

In [19]:
base_filters = 32
w_regularizer = 1e-4

In [20]:
model = tf.keras.Sequential()

# First layer
model.add(Conv2D(filters=base_filters, kernel_size=(3,3), kernel_regularizer=regularizers.l2(w_regularizer), padding='same', activation='relu', input_shape=(100,100,1)))
model.add(Activation('relu'))
model.add(BatchNormalization())

# Second layer
model.add(Conv2D(filters=2*base_filters, kernel_size=(3,3), kernel_regularizer=regularizers.l2(w_regularizer), padding='same', activation='relu'))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.2))

# Third layer
model.add(Conv2D(filters=2*base_filters, kernel_size=(3,3), kernel_regularizer=regularizers.l2(w_regularizer), padding='same', activation='relu'))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(Dropout(0.2))

# Fourth layer
model.add(Conv2D(filters=2*base_filters, kernel_size=(3,3), kernel_regularizer=regularizers.l2(w_regularizer), padding='same', activation='relu'))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.3))

# Fifth layer
model.add(Conv2D(filters=4*base_filters, kernel_size=(3,3), kernel_regularizer=regularizers.l2(w_regularizer), padding='same', activation='relu'))
model.add(Activation('relu'))
model.add(BatchNormalization())

# Sixth layer
model.add(Conv2D(filters=4*base_filters, kernel_size=(3,3), kernel_regularizer=regularizers.l2(w_regularizer), padding='same', activation='relu'))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.4))

# Flatten the output
model.add(Flatten())
model.add(Dense(16, activation='softmax'))

model.summary()


Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_6 (Conv2D)           (None, 100, 100, 32)      320       
                                                                 
 activation_6 (Activation)   (None, 100, 100, 32)      0         
                                                                 
 batch_normalization_6 (Batc  (None, 100, 100, 32)     128       
 hNormalization)                                                 
                                                                 
 conv2d_7 (Conv2D)           (None, 100, 100, 64)      18496     
                                                                 
 activation_7 (Activation)   (None, 100, 100, 64)      0         
                                                                 
 batch_normalization_7 (Batc  (None, 100, 100, 64)     256       
 hNormalization)                                      

In [21]:
model.compile(loss='categorical_crossentropy', optimizer='rmsprop', metrics=['accuracy'])

In [22]:
hist = model.fit(train_generator, steps_per_epoch=len(train_generator), epochs=200, verbose=1)

Epoch 1/200
40/40 [==============================] - 3s 37ms/step - loss: 11.6355 - accuracy: 0.0820
Epoch 2/200
40/40 [==============================] - 2s 37ms/step - loss: 4.9233 - accuracy: 0.0945
Epoch 3/200
40/40 [==============================] - 2s 39ms/step - loss: 4.0341 - accuracy: 0.1203
Epoch 4/200
40/40 [==============================] - 2s 39ms/step - loss: 3.5837 - accuracy: 0.1227
Epoch 5/200
40/40 [==============================] - 2s 39ms/step - loss: 3.2852 - accuracy: 0.1344
Epoch 6/200
40/40 [==============================] - 2s 40ms/step - loss: 3.1082 - accuracy: 0.1437
Epoch 7/200
40/40 [==============================] - 2s 41ms/step - loss: 2.9508 - accuracy: 0.1594
Epoch 8/200
40/40 [==============================] - 2s 41ms/step - loss: 2.7858 - accuracy: 0.1937
Epoch 9/200
40/40 [==============================] - 2s 41ms/step - loss: 2.7194 - accuracy: 0.1937
Epoch 10/200
40/40 [==============================] - 2s 41ms/step - loss: 2.6412 - accuracy: 0.195

In [23]:
# Assuming test_images and test_labels are your testing data and labels
test_loss, test_acc = model.evaluate(test_generator, verbose=1)

print('\nTest accuracy:', test_acc)
print('\nTest loss:', test_loss)

10/10 [==============================] - 0s 23ms/step - loss: 3.7933 - accuracy: 0.4500

Test accuracy: 0.44999998807907104

Test loss: 3.79325532913208


## Save model 

In [24]:
# Save the model
model.save('out/cnn2.keras')